In [1]:
from datetime import datetime, timedelta
import psycopg2
import pandas as pd
from datetime import datetime, timedelta
from bson import ObjectId
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from sshtunnel import SSHTunnelForwarder
from dbutils.pooled_db import PooledDB

ssh_host = 'api.trustyapp.io'
ssh_username = 'ubuntu'
ssh_pem_file = './ng-prod.pem'
database_host = '127.0.0.1'  # SSH 터널을 통해 localhost로 접근
database_name = 'trusty'
database_username = 'server'
database_password = 'ygFAhsyTMDzB21VdLTnoq7djxwjDNzuc9yPGukm73otXER8ihD'

server = SSHTunnelForwarder(
    (ssh_host, 22),  # SSH 서버 주소와 포트
    ssh_username=ssh_username,
    ssh_pkey=ssh_pem_file,  # Private key 파일 경로
    remote_bind_address=(database_host, 5432)  # 원격 MySQL 서버 주소와 포트
)

server.start()

conn = psycopg2.connect(
    host='127.0.0.1',  # localhost를 통해 SSH 터널로 접속
    user=database_username,
    password=database_password,
    database=database_name,
    port=server.local_bind_port  # sshtunnel이 자동으로 할당한 포트
)
cursor = conn.cursor()

In [21]:
conn.rollback()

qry = """
WITH cte AS (
    SELECT
        "userId",
        "status",
        -- date 타입은 그대로 사용
        "openedAt" AS open_date,
        "dueAt" AS due_date,
        "lastRepaymentAt" AS last_repay_date,
        -- numeric 컬럼은 COALESCE로 NULL → 0 처리
        COALESCE("loanAmount", 0) AS loan_amt
    FROM "credit_history"
),
grouped AS (
    SELECT
        "userId",
        -- 조기상환 플래그
        SUM(
            CASE
                WHEN "status" = 'performing'
                     AND open_date IS NOT NULL
                     AND due_date IS NOT NULL
                     AND last_repay_date IS NOT NULL
                     AND (last_repay_date - open_date) < 0.5 * (due_date - open_date)
                THEN 1 ELSE 0
            END
        ) AS early_repay_count,

        -- lost 상태 플래그
        SUM(
            CASE WHEN "status" = 'lost' THEN 1 ELSE 0 END
        ) AS lost_count,

        -- 조기상환 대출금액 평균
        AVG(
            CASE 
                WHEN "status" = 'performing'
                     AND open_date IS NOT NULL
                     AND due_date IS NOT NULL
                     AND last_repay_date IS NOT NULL
                     AND (last_repay_date - open_date) < 0.5 * (due_date - open_date)
                 THEN loan_amt
            END
        ) AS avg_early_loan_amt,

        -- lost 대출 최대금액
        MAX(
            CASE WHEN "status" = 'lost' THEN loan_amt END
        ) AS max_lost_loan_amt,

        -- 전체 건수
        COUNT(*) AS total_loan_count
    FROM cte
    GROUP BY "userId"
)
SELECT
    "userId",
    CASE WHEN (
       (CASE WHEN early_repay_count >= 3 THEN 1 ELSE 0 END)
       + (CASE WHEN (early_repay_count::float / NULLIF(total_loan_count, 0)) >= 0.3 THEN 1 ELSE 0 END)
       + (CASE WHEN lost_count >= 2 THEN 1 ELSE 0 END)
       + (CASE WHEN max_lost_loan_amt >= (avg_early_loan_amt * 2.5) THEN 1 ELSE 0 END)
    ) >= 3
    THEN 1 ELSE 0 END AS is_kkun,

    (CASE WHEN early_repay_count >= 3 THEN 1 ELSE 0 END) AS cond1,
    (CASE WHEN (early_repay_count::float / NULLIF(total_loan_count, 0)) >= 0.3 THEN 1 ELSE 0 END) AS cond2,
    (CASE WHEN lost_count >= 2 THEN 1 ELSE 0 END) AS cond3,
    (CASE WHEN max_lost_loan_amt >= (avg_early_loan_amt * 2.5) THEN 1 ELSE 0 END) AS cond4
FROM grouped;

"""
cursor.execute(qry)
result = cursor.fetchall()

# DataFrame 생성
df = pd.DataFrame(result, columns=[desc[0] for desc in cursor.description])
df.to_csv('./o1-8적용.csv', index=False, encoding='utf-8-sig')

# 결과 확인
print(df.head())

   userId  is_kkun  cond1  cond2  cond3  cond4
0     384        0      0      0      1      0
1     351        0      0      0      0      0
2     116        0      0      0      0      0
3      87        0      1      0      1      0
4     652        0      0      0      1      0
